# 第8章 文埋め込み

## 8.4 最近傍探索ライブラリ `Faiss` を使った検索

### 8.4.2 `Faiss`を利用した最近傍探索の実装

#### 準備

In [ ]:
!pip install datasets faiss-cpu scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 72.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.7 MB/s eta 0

#### データセットの読み込みと前処理

In [ ]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-paragraphsのリポジトリから
# Wikipediaの段落テキストのデータを読み込む
paragraph_dataset = load_dataset(
    "llm-book/jawiki-paragraphs", split="train"
)

Generating train split:   0%|          | 0/9668476 [00:00<?, ? examples/s]

Dataset jawiki-paragraphs downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___jawiki-paragraphs/default/1.0.0/0f2d7acd99ad7ae0615fd07442dbd1654d37c5d60a39fc720efe28acff3f86f8. Subsequent calls will reuse this data.


In [ ]:
# 段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [ ]:
from pprint import pprint

# 段落データの内容を確認する
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [ ]:
# 段落データのうち、各記事の最初の段落のみを使うようにする
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

Filter:   0%|          | 0/9668476 [00:00<?, ? examples/s]

In [ ]:
# フィルタリング後の段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 1339236
})


In [ ]:
# フィルタリング後の段落データの内容を確認する
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '10-94194440-0',
 'pageid': 10,
 'paragraph_index': 0,
 'revid': 94194440,
 'section': '__LEAD__',
 'text': '言語(げんご)は、狭義には「声による記号の体系」をいう。',
 'title': '言語'}


#### トークナイザとモデルの準備

Hugging Face Hubから読み込む場合

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Hugging Face Hubにアップロードされた
# 教師なしSimCSEのトークナイザとエンコーダを読み込む
model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

Google ドライブに保存したモデルを読み込む場合

In [ ]:
from google.colab import drive

drive.mount("drive")

Mounted at drive


In [ ]:
!cp -r drive/MyDrive/llm-book/outputs_unsup_simcse .

In [ ]:
from transformers import AutoModel, AutoTokenizer

# ディスクに保存された教師なしSimCSEのトークナイザとエンコーダを読み込む
model_path = "outputs_unsup_simcse/encoder"
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoder = AutoModel.from_pretrained(model_path)

共通の処理

In [ ]:
# 読み込んだモデルをGPUのメモリに移動させる
device = "cuda:0"
encoder = encoder.to(device)

#### モデルによる埋め込みの計算

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""
    # テキストにトークナイザを適用
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    # トークナイズされたテキストをベクトルに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(
                **tokenized_texts
            ).last_hidden_state[:, 0]

    # ベクトルをNumPyのarrayに変換
    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが1になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [ ]:
# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

Map:   0%|          | 0/1339236 [00:00<?, ? examples/s]

In [ ]:
# 埋め込みを付与した段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [ ]:
# 埋め込みを計算した段落データの内容を確認する
pprint(paragraph_dataset[0])

{'embeddings': [0.04253670945763588,
                -0.041921038180589676,
                -0.03232395276427269,
                0.01823267713189125,
                -0.06700421124696732,
                -0.060905277729034424,
                -0.0534023717045784,
                0.005872650071978569,
                0.005581581499427557,
                0.00042301995563320816,
                0.05438239127397537,
                -0.030172063037753105,
                -0.015410738065838814,
                -0.09762054681777954,
                0.031499966979026794,
                0.007067433558404446,
                0.004230297636240721,
                -0.018429215997457504,
                -0.07031217217445374,
                0.009732971899211407,
                0.006155171897262335,
                -0.03274840489029884,
                -0.008405999280512333,
                -0.023153288289904594,
                0.051212359219789505,
                0.04340992122888565,
        

In [ ]:
# 埋め込みを付与した段落データをディスクに保存する
paragraph_dataset.save_to_disk(
    "outputs_unsup_simcse/embedded_paragraphs"
)

Saving the dataset (0/10 shards):   0%|          | 0/1339236 [00:00<?, ? examples/s]

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

drive.mount("drive")

In [ ]:
# 保存された段落データをGoogleドライブのフォルダにコピーする
!cp -r outputs_unsup_simcse/embedded_paragraphs drive/MyDrive/llm-book/outputs_unsup_simcse

#### `Faiss` による最近傍探索を試す

In [ ]:
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定して空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データの"embeddings"フィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

  0%|          | 0/1340 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})

In [ ]:
query_text = "日本語は、主に日本で話されている言語である。"

# 最近傍探索を実行し、類似度上位10件の事例とスコアを取得する
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(
    "embeddings", embed_texts([query_text])[0], k=10
)
# 取得した事例の内容をスコアとともに表示する
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]
for score, title, text in zip(scores, titles, texts):
    print(score, title, text)

0.78345203 日本の言語 日本の言語(にほんのげんご)は、日本の国土で使用されている言語について記述する。日本#言語も参照。
0.75877357 日本語教育 日本語教育(にほんごきょういく)とは、外国語としての日本語、第二言語としての日本語についての教育の総称である。
0.7494176 日本語学 日本語学(にほんごがく)とは、日本語を研究の対象とする学問である。
0.74729466 日本語 日本語(にほんご、にっぽんご、英語: Japanese)は、日本国内や、かつての日本領だった国、そして国外移民や移住者を含む日本人同士の間で使用されている言語。日本は法令によって公用語を規定していないが、法令その他の公用文は全て日本語で記述され、各種法令において日本語を用いることが規定され、学校教育においては「国語」の教科として学習を行う等、事実上、日本国内において唯一の公用語となっている。
0.7045407 国語 (教科) 国語(こくご、英: Japanese Language)は、日本の学校教育における教科の一つ。
0.7029643 和製英語 和製英語(わせいえいご)は、日本語の中で使われる和製外来語の一つで、日本で日本人により作られた、英語の言葉や英語に似ている言葉(固有名詞や商品名などを除く)である。英語圏では別表現をするために理解されなかったり、もしくは、全く異なった解釈をされたりする場合がある。
0.6956495 口語 口語(こうご)とは、普通の日常的な生活の中での会話で用いられる言葉遣いのことである。書記言語で使われる文語と違い、方言と呼ばれる地域差や社会階層などによる言語変種が応じやすく、これらと共通語などを使い分ける状態はダイグロシアと呼ばれる。
0.6944481 ジャパン ジャパン(英語: Japan)は、英語で日本を意味する単語。
0.6911353 日本語学科 日本語学科(にほんごがっか)とは、日本語を教育研究することを目的として大学や専門学校などの高等教育機関に置かれる学科の名称である。
0.6908301 日本語学校 日本語学校(にほんごがっこう)とは、主に日本語を母語としない者を対象として、第二言語・外国語としての日本語教育を実施する機関。日本国内外に存在している。
